In [1]:
import torch.nn.functional as F
import torch.nn as nn

class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        """
            input   - (3, 32, 32)
            block 1 - (32, 32, 32)
            maxpool - (32, 16, 16)
            block 2 - (64, 16, 16)
            maxpool - (64, 8, 8)
            block 3 - (128, 8, 8)
            maxpool - (128, 4, 4)
            block 4 - (128, 4, 4)
            avgpool - (128, 1, 1), reshpe to (128,)
            fc      - (128,) -> (10,)

        """
        # block 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        # block 2
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # block 3
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # block 4
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, 10)

    def forward(self, x):

        # block 1
        x = F.relu(self.conv1(x))
        x = F.relu(self.bn1(self.conv2(x)))

        # maxpool
        x = F.max_pool2d(x, 2)

        # block 2
        x = F.relu(self.conv3(x))
        x = F.relu(self.bn2(self.conv4(x)))

        # maxpool
        x = F.max_pool2d(x, 2)

        # block 3
        x = F.relu(self.conv5(x))
        x = F.relu(self.bn3(self.conv6(x)))

        # maxpool
        x = F.max_pool2d(x, 2)

        # block 4
        x = F.relu(self.conv7(x))
        x = F.relu(self.bn4(self.conv8(x)))

        # avgpool and reshape to 1D
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        # fc
        x = self.fc(x)

        return x

In [2]:
from torchvision import models
import torch
import cv2
import numpy as np
from scipy.optimize import differential_evolution
import torch.nn as nn
from torch.autograd import Variable
# from model import BasicCNN

import argparse

In [3]:
cifar10_class_names = {0: 'airplane', 1: 'automobile',
                       2: 'bird', 3: 'cat', 4: 'deer',
                       5: 'dog', 6: 'frog', 7: 'horse',
                       8: 'ship', 9: 'truck'}

In [4]:
image_path = "./airplane.png"                  #Path to image
d = 1                                          #Number of pixels to change
iters = 600                                    #Number of Iterators
popsize = 10                                   #Population size
model_path = "./cifar10_basiccnn.pth.tar"      #Path to trained model

In [5]:
def nothing(x):
    pass

In [6]:
# load image and reshape to (3, 224, 224) and RGB (not BGR)
orig = cv2.imread(image_path)[..., ::-1]
orig = cv2.resize(orig, (32, 32))
img = orig.copy()
shape = orig.shape

In [7]:
def preprocess(img):
    img = img.astype(np.float32)
    img /= 255.0
    img = img.transpose(2, 0, 1)
    return img

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [8]:
model = BasicCNN()
saved = torch.load(model_path, map_location='cpu')
model.load_state_dict(saved['state_dict'])
model.eval()

inp = Variable(torch.from_numpy(preprocess(img)).float().unsqueeze(0))
prob_orig = softmax(model(inp).data.numpy()[0])
pred_orig = np.argmax(prob_orig)
print('Prediction before attack: %s' %(cifar10_class_names[pred_orig]))
print('Probability: %f' %(prob_orig[pred_orig]))
print()

Prediction before attack: airplane
Probability: 0.544428



In [9]:
def perturb(x):
    adv_img = img.copy()

    # calculate pixel locations and values
    pixs = np.array(np.split(x, len(x)/5)).astype(int)
    loc = (pixs[:, 0], pixs[:,1])
    val = pixs[:, 2:]
    adv_img[loc] = val

    return adv_img

def optimize(x):
    adv_img = perturb(x)

    inp = Variable(torch.from_numpy(preprocess(adv_img)).float().unsqueeze(0))
    out = model(inp)
    prob = softmax(out.data.numpy()[0])

    return prob[pred_orig]

pred_adv = 0
prob_adv = 0

In [10]:
def callback(x, convergence):
    global pred_adv, prob_adv
    adv_img = perturb(x)

    inp = Variable(torch.from_numpy(preprocess(adv_img)).float().unsqueeze(0))
    out = model(inp)
    prob = softmax(out.data.numpy()[0])

    pred_adv = np.argmax(prob)
    prob_adv = prob[pred_adv]
    if pred_adv != pred_orig and prob_adv >= 0.9:
        print('Attack successful..')
        print('Prob [%s]: %f' %(cifar10_class_names[pred_adv], prob_adv))
        print()
        return True
    else:
        print('Prob [%s]: %f' %(cifar10_class_names[pred_orig], prob[pred_orig]))


def scale(x, scale=5):
    return cv2.resize(x, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)


In [ ]:
while True:
    bounds = [(0, shape[0]-1), (0, shape[1]), (0, 255), (0, 255), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)
    inp = Variable(torch.from_numpy(preprocess(adv_img)).float().unsqueeze(0))
    out = model(inp)
    prob = softmax(out.data.numpy()[0])
    print('Prob [%s]: %f --> Prob[%s]: %f' %(cifar10_class_names[pred_orig], prob_orig[pred_orig], cifar10_class_names[pred_adv], prob_adv))

    cv2.imshow('adversarial image', scale(adv_img[..., ::-1]))

    key = 0
    while True:
        print("Press 'esc' to exit, 'space' to re-run..", end="\r")
        key = cv2.waitKey(100) & 0xFF
        if key == 27:
            break
        elif key == ord('s'):
            cv2.imwrite('adv_img.png', scale(adv_img[..., ::-1]))
        elif key == 32:
            break
    if key == 27:
        break
cv2.destroyAllWindows()

Prob [airplane]: 0.192419
Prob [airplane]: 0.192419
Prob [airplane]: 0.135264
Prob [airplane]: 0.135264
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.097694
Prob [airplane]: 0.094966
Attack successful..
Prob [bird]: 0.908650

Prob [airplane]: 0.544428 --> Prob[bird]: 0.908650
